<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

## Загрузка данных

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Данные представлены в 5 колонках, 4 из них признаки и один целевой признак `Страховые выплаты`. Пропуски в данных отсутсвуют.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    - a. Изменится. Приведите примеры матриц.
    - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков $X1$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X1 = X * P
$$

Подставим новое значение X1 в формулу $w_1$:

$$
w_1 = ((X P)^T X P)^{-1} (X P)^T y
$$

Раскроем первое произведение $ (XP)^T $:

$$
w_1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w_1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$

Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Можно заметить, что справа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

Подставим новое значение весов  𝑤1  в формулу для предсказаний линейной регрессии.
Предсказания модели расчитываются по формуле:

$$
a = Xw
$$

Подставим в эту формулу значения $X1$ и $w_1$ для того, чтобы на их основе вычислить предсказания модели $a1$:

$$
a1 = X1w1 = XPP^{-1} w
$$

Так как $P$ по условию обратимая, то произведение $PP^{-1}$ снова равно $E$:

$$
a1 = XPP^{-1} w = X E w = X w = a
$$

Мы доказали, что предсказания $a1$ для матрицы признаков, умноженных на обратимую матрицу $P$ равны предсказаниям $a$.<br>
Следовательно, отвечая на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу - нет, не изменится. В данном случае, параметры линейной регрессии в исходной задаче $w$ и в преобразованной $w1$ связаны следующим образом:

$$
w_1 = P^{-1} w
$$

**Вывод:**
При умножении признаков на обратимую матрицу, предсказания модели не поменяется, но поменяются веса для новых признаков. Придется переобучать модель на новых признаках.

## Алгоритм преобразования

Чтобы защитить данные и не потерять качество модели машинного обучения, нужно умножить признаки на обратимую матрицу $P$. 
Для этого алгоритм преобразования данных должен содержать процесс создания подобной матрицы $P$ и ее умножения на матрицу признаков.

In [9]:
# Создами рандомную матрицу, такого же размера как и исходная 
random_matrix = np.random.randint(100,size = (4, 4))
random_matrix

array([[34, 67, 92, 28],
       [57, 39, 54, 38],
       [ 7, 44, 24, 19],
       [32, 33, 29, 74]])

In [10]:
# Проверим на обратимость, если матрица не обратима будет ошибка.
matrix_inverted = np.linalg.inv(random_matrix)
matrix_inverted

array([[-0.0178678 ,  0.03447281,  0.00601714, -0.01248641],
       [-0.01254416,  0.00804061,  0.04240428, -0.01027011],
       [ 0.02560857, -0.01472345, -0.03015454,  0.00561334],
       [ 0.00328484, -0.01272284, -0.0096947 ,  0.02129313]])

Из формул известно, что предсказания модели при преобразовании не поменяются, а значит качество модели тоже не должно меняться, так как качество модели - это производная от предсказаний (которые одинаковы до и после преобразований) и целевого признака.

Подкрепим теоритические рассуждения примером на цифрах из таблицы. Для этого самостоятельно расчитаем вектора предсказаний $a$ и $a1$ по формулам и сравним их между собой.

In [13]:
# Выделим признаки и целевой признак.
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [17]:
#Маштабируем
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

In [21]:
#Расчитаем веса w для исходных признаков (масштабированных) по стандартной формуле w = (X^T X)^{-1} X^T y
w = np.linalg.inv(features_scaled.T.dot(features_scaled)).dot(features_scaled.T).dot(target)

In [23]:
#Расчитаем вектор предсказаний по исходным признакам по формуле 𝑎=𝑋𝑤
a = features_scaled @ w

In [25]:
#Проведем преобразование признаков:
features_scaled_p = features_scaled @ random_matrix

In [26]:
#Расчитаем веса  𝑤1  для преобразованных признаков по стандартной формуле  𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦
w1 = np.linalg.inv(features_scaled_p.T.dot(features_scaled_p)).dot(features_scaled_p.T).dot(target)

In [27]:
#Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎=𝑋𝑤
a1 = features_scaled_p @ w1

In [28]:
#Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1 :
diff = a - a1

In [29]:
#Просуммируем все элементы вектора diff, чтобы убедиться, что расхождения минимальны.
diff.sum()

-3.003120077919516e-14

Исходя из полученных данных мы видим, что сумма расхождений совсем маленькая. Это значит 𝑎 равен 𝑎1. Мы доказали, что преобразование не приводит к разным предсказаниям модели и не ухудшает качество модели.

## Проверка алгоритма

In [33]:
# Создадим модель линенйной решресии для исходных данных и посмотрим на ее метрику R2
model = LinearRegression(normalize = True)
model.fit(features,target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score на исходных данных: ',r2_score(target, predictions))

MSE Score: 0.12334688941710856
R2_Score на исходных данных:  0.4249455028666802


In [38]:
# Создадим модель линенйной решресии для преобразованных данных и посмотрим на ее метрику R2 и сравним 
features_matrix = features_scaled @ random_matrix
model.fit(features_matrix, target)
predictions_matrix = model.predict(features_matrix)
mse = mean_squared_error(target, predictions)
print('MSE Score:', mse)
print('R2_Score для преобразованных  признаков: ',r2_score(target, predictions_matrix))

MSE Score: 0.1233468894171086
R2_Score для преобразованных  признаков:  0.42494550286668


**Вывод:** 
Исходя из показателей метрик, мы защитили и сохранили данные и не повляли на качество модели.